# Feature Engineering

In [6]:
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

In [25]:
class MercariFeatureEngineering(object):
    
    def __init__(self, filepath, delimiter=','):
        self.df = pd.read_csv(filepath, delimiter=delimiter)
        #self.stop_words = set(stopwords.words('english'))
        self.alphabet = set(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j',
                             'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't',
                             'u', 'v', 'w', 'x', 'y', 'z'])
        self.rf = None
    
    def fill_na(self, column_name, new_col, fill_with):
        self.df[new_col] = self.df[column_name].isnull().astype(int)
        self.df[column_name] = self.df[column_name].fillna(fill_with)
    
    def split_categories(self, column_name, split_on):
        top, middle, bottom = [], [], []
        for i, row in self.df.iterrows():
            hierarchy_string = row[column_name]
            hierarchy_list = hierarchy_string.split(split_on)
            top.append(hierarchy_list[0])
            middle.append(hierarchy_list[1])
            bottom.append(hierarchy_list[2])
        self.df['cat_top'] = top
        self.df['cat_mid'] = middle
        self.df['cat_bot'] = bottom
    
    def clean_column(self, brand_name):
        word = brand_name.lower()
        word = ''.join([letter for letter in word if letter in self.alphabet])
        return word
    
    def drop_rows_with_value(self, column, value):
        self.df = self.df[self.df[column] != value]
    
    def apply_func(self, new_name, from_col, func):
        self.df[new_name] = self.df[from_col].apply(lambda x: func(x))
    
    def create_categorical_labels(self, column_name, new_col):
        self.df[new_col] = LabelEncoder().fit_transform(self.df[column_name])
    
    def engineer_features(self):
        if 'price' in self.df.columns:
            self.drop_rows_with_value('price', 0)
        self.fill_na('category_name', 'cat_Was_null', 'None/None/None')
        self.fill_na('brand_name', 'brand_was_null', 'no_label')
        self.fill_na('item_description', 'desc_was_null', 'No description')
        print ('All Nulls Filled, New Binary Columns Created!')
        self.split_categories('category_name', '/')
        print('Categories Split!')
        self.apply_func('brand_name', 'brand_name', self.clean_column)
        self.apply_func('category_top', 'cat_top', self.clean_column)
        self.apply_func('category_middle', 'cat_mid', self.clean_column)
        self.apply_func('category_bottom', 'cat_bot', self.clean_column)
        print('Cleaned Columns')
        self.create_categorical_labels('brand_name', 'brand_numeric')
        self.create_categorical_labels('cat_top', 'cat_top_numeric')
        self.create_categorical_labels('cat_mid', 'cat_mid_numeric')
        self.create_categorical_labels('cat_bot', 'cat_bot_numeric')
        print('Created Categorical Labels')
        

In [26]:
start = datetime.now()
train = MercariFeatureEngineering('data/train.tsv', delimiter='\t')
train.engineer_features()
end = datetime.now()
print end - start

All Nulls Filled, New Binary Columns Created!
Categories Split!
Cleaned Columns
Created Categorical Labels
0:02:11.749599


In [27]:
train.df.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,cat_Was_null,brand_was_null,...,cat_top,cat_mid,cat_bot,category_top,category_middle,category_bottom,brand_numeric,cat_top_numeric,cat_mid_numeric,cat_bot_numeric
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,nolabel,10.0,1,No description yet,0,1,...,Men,Tops,T-shirts,men,tops,tshirts,3093,5,103,763
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,razer,52.0,0,This keyboard is in great condition and works ...,0,0,...,Electronics,Computers & Tablets,Components & Parts,electronics,computerstablets,componentsparts,3576,1,30,212
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,target,10.0,1,Adorable top with a hint of lace and a key hol...,0,0,...,Women,Tops & Blouses,Blouse,women,topsblouses,blouse,4193,10,104,94
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,nolabel,35.0,1,New with tags. Leather horses. Retail for [rm]...,0,1,...,Home,Home Décor,Home Décor Accents,home,homedcor,homedcoraccents,3093,3,55,404
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,nolabel,44.0,0,Complete with certificate of authenticity,0,1,...,Women,Jewelry,Necklaces,women,jewelry,necklaces,3093,10,58,536
